# Cleaning Demographic Data

To see the features selected to be included in the cleaned demographic data, scroll to the bottom of this notebook to where the dictionary "meta_features" is printed. The cleaned data contains those features, which are written to a csv file named "cleaned_demographic_features.csv" in the data folder.

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [4]:
demCensus=pd.read_csv('data/allDemographics2000.csv')
demCensus['GEO.id2'] = demCensus['GEO.id2'].astype(str)
demCensus['GEO.id2'] = [(5-len(zipcode))*'0' + zipcode if len(zipcode) < 5 else zipcode for zipcode in demCensus['GEO.id2']]
demCensus.columns = demCensus.iloc[0].values
demCensus.head()

,Id,00Id2,Geography,Number; Total population,Percent; Total population,Number; Total population - SEX AND AGE - Male,Percent; Total population - SEX AND AGE - Male,Number; Total population - SEX AND AGE - Female,Percent; Total population - SEX AND AGE - Female,Number; Total population - SEX AND AGE - Under 5 years,Percent; Total population - SEX AND AGE - Under 5 years,Number; Total population - SEX AND AGE - 5 to 9 years,Percent; Total population - SEX AND AGE - 5 to 9 years,Number; Total population - SEX AND AGE - 10 to 14 years,Percent; Total population - SEX AND AGE - 10 to 14 years,Number; Total population - SEX AND AGE - 15 to 19 years,Percent; Total population - SEX AND AGE - 15 to 19 years,Number; Total population - SEX AND AGE - 20 to 24 years,Percent; Total population - SEX AND AGE - 20 to 24 years,Number; Total population - SEX AND AGE - 25 to 34 years,Percent; Total population - SEX AND AGE - 25 to 34 years,Number; Total population - SEX AND AGE - 35 to 44 years,Percent; Total population - SEX AND AGE - 35 to 44 years,Number; Total population - SEX AND AGE - 45 to 54 years,Percent; Total population - SEX AND AGE - 45 to 54 years,Number; Total population - SEX AND AGE - 55 to 59 years,Percent; Total population - SEX AND AGE - 55 to 59 years,Number; Total population - SEX AND AGE - 60 to 64 years,Percent; Total population - SEX AND AGE - 60 to 64 years,Number; Total population - SEX AND AGE - 65 to 74 years,Percent; Total population - SEX AND AGE - 65 to 74 years,Number; Total population - SEX AND AGE - 75 to 84 years,Percent; Total population - SEX AND AGE - 75 to 84 years,Number; Total population - SEX AND AGE - 85 years and over,Percent; Total population - SEX AND AGE - 85 years and over,Number; Total population - SEX AND AGE - Median age (years),Percent; Total population - SEX AND AGE - Median age (years),Number; Total population - SEX AND AGE - 18 years and over,Percent; Total population - SEX AND AGE - 18 years and over,Number; Total population - SEX AND AGE - 18 years and over - Male,Percent; Total population - SEX AND AGE - 18 years and over - Male,Number; Total population - SEX AND AGE - 18 years and over - Female,Percent; Total population - SEX AND AGE - 18 years and over - Female,Number; Total population - SEX AND AGE - 21 years and over,Percent; Total population - SEX AND AGE - 21 years and over,Number; Total population - SEX AND AGE - 62 years and over,Percent; Total population - SEX AND AGE - 62 years and over,Number; Total population - SEX AND AGE - 65 years and over,Percent; Total population - SEX AND AGE - 65 years and over,Number; Total population - SEX AND AGE - 65 years and over - Male,...,Number; HOUSEHOLDS BY TYPE - Total households,Percent; HOUSEHOLDS BY TYPE - Total households,Number; HOUSEHOLDS BY TYPE - Total households - Family households (families),Percent; HOUSEHOLDS BY TYPE - Total households - Family households (families),Number; HOUSEHOLDS BY TYPE - Total households - Family households (families) - With own children under 18 years,Percent; HOUSEHOLDS BY TYPE - Total households - Family households (families) - With own children under 18 years,Number; HOUSEHOLDS BY TYPE - Total households - Family households (families) - Married-couple family,Percent; HOUSEHOLDS BY TYPE - Total households - Family households (families) - Married-couple family,Number; HOUSEHOLDS BY TYPE - Total households - Family households (families) - Married-couple family - With own children under 18 years,Percent; HOUSEHOLDS BY TYPE - Total households - Family households (families) - Married-couple family - With own children under 18 years,"Number; HOUSEHOLDS BY TYPE - Total households - Family households (families) - Female householder, no husband present","Percent; HOUSEHOLDS BY TYPE - Total households - Family households (families) - Female householder, no husband present","Number; HOUSEHOLDS BY TYPE - Total households - Family households (families) - Female householder, no husband present - With own children unde

In [3]:
demCensus.shape

(33179, 195)

In [17]:
#exploring the different features in this table
prelimFeatures = demCensus.loc[0]
for i in prelimFeatures:
    print i

Id
00Id2
Geography
Number; Total population
Percent; Total population
Number; Total population - SEX AND AGE - Male
Percent; Total population - SEX AND AGE - Male
Number; Total population - SEX AND AGE - Female
Percent; Total population - SEX AND AGE - Female
Number; Total population - SEX AND AGE - Under 5 years
Percent; Total population - SEX AND AGE - Under 5 years
Number; Total population - SEX AND AGE - 5 to 9 years
Percent; Total population - SEX AND AGE - 5 to 9 years
Number; Total population - SEX AND AGE - 10 to 14 years
Percent; Total population - SEX AND AGE - 10 to 14 years
Number; Total population - SEX AND AGE - 15 to 19 years
Percent; Total population - SEX AND AGE - 15 to 19 years
Number; Total population - SEX AND AGE - 20 to 24 years
Percent; Total population - SEX AND AGE - 20 to 24 years
Number; Total population - SEX AND AGE - 25 to 34 years
Percent; Total population - SEX AND AGE - 25 to 34 years
Number; Total population - SEX AND AGE - 35 to 44 years
Percent; Tot

In [20]:
#keep these non-percentage data points: id, zipcode, raw population numbers
#the rest of the cleaned data will be percentages
features = demCensus.filter(regex='GEO').ix[1:, :]
features["A001"] = demCensus["HD01_S001"][1:].astype(float)

#start a supplementary table of metadata
meta_features = {"A001": "Number; SEX AND AGE - Total population"}

features.head()

,GEO.id,GEO.id2,GEO.display-label,A001
1,8600000US00601,00601,ZCTA5 00601,19143
2,8600000US00602,00602,ZCTA5 00602,42042
3,8600000US00603,00603,ZCTA5 00603,55530
4,8600000US00604,00604,ZCTA5 00604,3923
5,8600000US00606,00606,ZCTA5 00606,6449


In [ ]:
#Clean up all strings and convert to float
#Replace '(X)' strings with NaN
#Get rid of the first column of 100% values 

percent_features_num = demCensus.filter(regex='HD02').ix[1:, 1:].replace(to_replace='(X)', value=np.nan, regex=True)
totalpop_agegroups = percent_features_num.astype(float)
percent_features = demCensus.filter(regex='HD02').ix[:0, 1:].append(totalpop_agegroups)

raw_features = demCensus.filter(regex='HD01').ix[1:, 1:].replace(to_replace='(X)', value=np.nan, regex=True)\
    .astype(float)

In [ ]:
percent_features.head()

In [ ]:
#function for aggregating data

def aggregate_data(start, end, interval, df, larger_df, category, gender, lst):
    #create temporary dataframe with correct length
    new_df = demCensus.filter(regex='HD02').ix[1:, :1]
    new_df = new_df.drop("HD02_S001", 1)
    length = len(df.columns)
    
    #aggregate two columns to get percentages for age groups of 10 years
    for i in range(start, end, interval):    
        first_col = 'HD02_S0' + ('%02d' % (i+2))
        second_col = 'HD02_S0' + ('%02d' % (i+3))
        col_name = "A0" + ('%02d' % ((i-start)/interval + length - 2))
        new_df[col_name] = larger_df[first_col] + larger_df[second_col]
    
    #update feature names
    for i in range(len(lst)):
        col_name = "A0" + ('%02d' % (i+length-2))
        meta_features[col_name] = "Percent; " + category + " - " + gender + " population - " + lst[i]
    
    #update external dataframe with results (this is cumulative)
    for column in new_df:
        df[column] = new_df[column]

    return df

In [ ]:
#get aggregated age data: do this section as a whole because these steps are cumulative

#set up parameters
lst = ["Under 10 years", 
       "10 to 19 years", 
       "20 to 29 years", 
       "30 to 39 years", 
       "40 to 49 years", 
       "50 to 59 years", 
       "60 to 69 years", 
       "70 to 79 years", 
       "Over 80 years"]

#getting data for total population
df = aggregate_data(0, 18, 2, features, percent_features, "SEX AND AGE", "Total", lst)

#getting data for male population
next_colname = "A0" + ('%02d' % (len(df.columns)-2))
df[next_colname] = percent_features["HD02_S026"] 
meta_features[next_colname] = percent_features["HD02_S026"][0]
df2 = aggregate_data(25, 43, 2, df, percent_features, "SEX AND AGE", "Male", lst)

#getting data for female population
next_colname = "A0" + ('%02d' % (len(df.columns)-2))
df2[next_colname] = percent_features["HD02_S051"] 
meta_features[next_colname] = percent_features["HD02_S051"][0]
df3 = aggregate_data(50, 68, 2, df2, percent_features, "SEX AND AGE", "Female", lst)

df3.head() #this dataframe has aggregated age data

In [ ]:
#get aggregated race data: also do this section as a whole because these steps are cumulative

#get White, Black or African American, American Indian and Alaska Native and Asian race data
for i in range(0,4):
    next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
    df3[next_colname] = percent_features["HD02_S0" + str(78 + i)][1:] 
    meta_features[next_colname] = percent_features["HD02_S0" + str(78 + i)][0]

#get Hawaiian and other Pacific Islander data
next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
df3[next_colname] = percent_features["HD02_S089"][1:] 
meta_features[next_colname] = percent_features["HD02_S089"][0]

#get Hispanic data
next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
df3[next_colname] = percent_features["HD02_S107"][1:]
meta_features[next_colname] = percent_features["HD02_S107"][0]

df3.head() #this dataframe now has aggregated age and race data

In [ ]:
#get household data
#can update housing features here if we want to add more from the original data
lst = ["HD02_S151", "HD02_S155", "HD02_S156", "HD02_S157", "HD02_S158", "HD02_S160", 
       "HD02_S161", "HD02_S162", "HD02_S163", "HD02_S164", "HD02_S165",
       "HD02_S166", "HD02_S170", "HD02_S171", "HD02_S172", "HD02_S174"]
lst_raw = ["HD01_S167", "HD01_S168", "HD01_S169", "HD01_S186"]

#percentage data points
for i in lst:
    next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
    df3[next_colname] = percent_features[i][1:] 
    meta_features[next_colname] = percent_features[i][0]

#raw data points (e.g. average household size)
for i in lst_raw:
    next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
    df3[next_colname] = raw_features[i] 
    meta_features[next_colname] = demCensus[i][0]
    
df3.head()

In [ ]:
#see all the feature names for the dataframe
for i in sorted(meta_features):
    print i, meta_features[i]

In [ ]:
df3['GEO.id2']=df3['GEO.id2'].astype(str)
df3.to_pickle("data/cleaned_demographic_features_2010.pkl")